In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs
import csv

In [2]:
import time
import random

In [3]:
def random_sleep():
    time.sleep(random.randint(2, 4))

In [ ]:
def read_page(): #creation d'un generateur
    i = 1
    while True:
        
        r = rq.get('https://www.autoplus.fr/prix-neuf/?page='+str(i)+'#advanced-filters') # recup du contenu
        
        random_sleep() # pause entre 2, 4 secs
        
        soup = bs(r.content, 'html.parser') # conversion du text brute en bs4 plus facile a parser
        trs = soup.table.tbody.find_all('tr') # recup des trs du tableau en html
        for tr in trs: # dans chaque tr il y a des tds
            values = [x.get_text(strip=True).replace('\xa0', '') for x in tr.find_all('td')] # extraction des infos depuis les tds
            row = dict(zip(KEYS, values)) # convertir les resultat extrait d'une liste vers un dictionaire
            yield row # Retoune un dictionaire
        i = i + 1
        print(i, end='\r')
        if soup.find(class_='main-pagination').find('li', class_='arrow right disabled') is not None:
            break # si il n y a plus de page suivante on arrete le programme

In [ ]:
# Creation d'un csv vierge
KEYS = ['name', 'version', 'energy', 'transmission', 'tax_horsepower', 'date_entry', 'bonus/malus', 'starting_price']
with open('cars_price.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=KEYS)
    writer.writeheader()

In [ ]:
# Initialisation du generateur
scraper = read_page()

In [ ]:
for row in scraper:
    # chaque row est un dictionaire qu'on va ecrire dans une nouvelle ligne de notre csv
    # ceci permet de ne pas saturé la ram en ecrivant chaque nouvelle ligne en physique et passé la ligne suivante
    with open('cars_price.csv', 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=KEYS)
        writer.writerow(row)